In [ ]:
import logging
from pathlib import Path
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np

USE_GPU = False

if USE_GPU:
    import manage_gpus as gpl

    gpl.get_gpu_lock()
import activation_learner
from pytorch_nmf import BetaDivergence
import param_estimator
import plot
from mixes.unmixdb import UnmixDB
from mixes.brabants import Brabants
import carve

In [ ]:
# hyperparams

FS = 22050
HOP_SIZES = [4, 2, 1, 0.5]
OVERLAP = 8
NMELS = 128
SPEC_POWER = 2
DIVERGENCE = BetaDivergence(0)
GAIN_ESTOR = param_estimator.GainEstimator.SUM
WARP_ESTOR = param_estimator.WarpEstimator.ARGMAX
LOW_POWER_THRESHOLD = 1e-2
CARVE_THRESHOLD = 1e-6
CARVE_BLUR_SIZE = 3
CARVE_MIN_DURATION = 10
CARVE_MAX_SLOPE = 1.5
# stop conditions
DLOSS_MIN = 1e-10
ITER_MAX = 5000
# paths
# RESULTS_DIR = Path("/data5/anasynth_nonbp/andre/reverse-dj-mix/results")
RESULTS_DIR = Path("/home/etiandre/stage/results/")
# UNMIXDB_PATH = Path("/data2/anasynth_nonbp/schwarz/abc-dj/data/unmixdb-zenodo")
DATASET_PATH = Path("/home/etiandre/stage/datasets/unmixdb/")
# DATASET_PATH = Path("/home/etiandre/stage/datasets/dj_mix_ground_truth_extractor_dataset")
#############################

In [ ]:
dataset = UnmixDB(DATASET_PATH) 
# dataset = SyntheticDB()
# mix = dataset.get_mix("set044mix3-none-none-15.mp3")
mix = dataset.mixes[0]
logger = logging.getLogger()
logging.basicConfig()
logger.setLevel(logging.INFO)
logger.info(mix.name)

inputs = [track.audio for track in mix.tracks] + [mix.audio]

In [ ]:
learner, loss_history = activation_learner.multistage(
    inputs,
    FS,
    hops=HOP_SIZES,
    overlap=OVERLAP,
    nmels=NMELS,
    low_power_threshold=LOW_POWER_THRESHOLD,
    spec_power=SPEC_POWER,
    divergence=DIVERGENCE,
    iter_max=ITER_MAX,
    dloss_min=DLOSS_MIN,
    carve_threshold=CARVE_THRESHOLD,
    carve_blur_size=CARVE_BLUR_SIZE,
    carve_min_duration=CARVE_MIN_DURATION,
    carve_max_slope=CARVE_MAX_SLOPE,
    doplot=True,
)

In [ ]:
# get ground truth
tau = np.arange(0, learner.V.shape[1]) * HOP_SIZES[-1]
real_gain = mix.gain(tau)
real_warp = mix.warp(tau)

# estimate gain
logger.info(f"Estimating gain with method {GAIN_ESTOR}")
est_gain = GAIN_ESTOR(learner.H, learner.split_idx, SPEC_POWER)

# estimate warp
logger.info(f"Estimating warp with method {WARP_ESTOR}")
est_warp = WARP_ESTOR(learner.H, learner.split_idx, HOP_SIZES[-1])

gain_err = param_estimator.error(est_gain, real_gain)
warp_err = param_estimator.error(est_warp, real_warp)

plt.figure(figsize=(16,16))
plot.plot_H(learner.H.detach().numpy())

plt.figure(figsize=(16,8))
plot.plot_gain(tau, est_gain, real_gain)
plt.title(f"Gain err={gain_err:.2e}")
plt.show()

plt.figure(figsize=(16,8))
plot.plot_warp(tau, est_warp, real_warp)
plt.title(f"Warp err={warp_err:.2e}")
plt.show()


In [ ]:
plt.plot(abs(est_gain - real_gain))